<a href="https://colab.research.google.com/github/MikeRoHer96/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load the dataset from the cloud URL
url = ("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df = pd.read_csv(url)

# Display the first few rows to ensure the data is loaded correctly
df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
# Step 2: Drop columns that are not needed (EIN, NAME)
df = df.drop(columns=['EIN', 'NAME'])

# Step 3: Identify the target and feature variables
# The target variable is IS_SUCCESSFUL, and the features are all other columns
y = df['IS_SUCCESSFUL']
X = df.drop(columns=['IS_SUCCESSFUL'])

# Step 4: Check the number of unique values for each column
print(X.nunique())

# Step 5: Handle rare categories in categorical variables
# Let's first check the value counts for columns with more than 10 unique values
for column in X.select_dtypes(include=['object']).columns:
    print(f"{column} value counts:")
    print(X[column].value_counts())

# Replace rare categories with 'Other'
for column in X.select_dtypes(include=['object']).columns:
    value_counts = X[column].value_counts()
    threshold = 100  # Set a threshold to define "rare" categories
    rare_categories = value_counts[value_counts < threshold].index
    X[column] = X[column].replace(rare_categories, 'Other')

# Step 6: Use pd.get_dummies to encode categorical variables
X_encoded = pd.get_dummies(X)

# Step 7: Split the data into features (X) and target (y)
# We already separated 'IS_SUCCESSFUL' into 'y' and the rest into 'X'

# Step 8: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25, random_state=42)

# Step 9: Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Check the first few rows of the scaled data
print(X_train_scaled[:5])

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
dtype: int64
APPLICATION_TYPE value counts:
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64
AFFILIATION value counts:
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: count, dtype: int64
CLASSIFICATION value counts:
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1

In [19]:
# Define the model
model = Sequential()

# Input layer and first hidden layer with 128 neurons and 'relu' activation
model.add(Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer with 64 neurons and 'relu' activation
model.add(Dense(units=64, activation='relu'))

# Third hidden layer with 32 neurons and 'relu' activation (optional but can help with learning)
model.add(Dense(units=32, activation='relu'))

# Output layer with a single neuron for binary classification using 'sigmoid' activation
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 128)                 │           6,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,513 (64.50 KB)

 Trainable params: 16,513 (64.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(Dense(units=64, activation='relu'))

# Third hidden layer
nn.add(Dense(units=32, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn.fit(X_train_scaled, y_train, epochs=10, batch_size=64, validation_data=(X_test_scaled, y_test))

# Save the model to an HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')


Epoch 1/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7207 - loss: 0.5801 - val_accuracy: 0.7293 - val_loss: 0.5576
Epoch 2/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7264 - loss: 0.5525 - val_accuracy: 0.7300 - val_loss: 0.5523
Epoch 3/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7306 - loss: 0.5488 - val_accuracy: 0.7289 - val_loss: 0.5542
Epoch 4/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7319 - loss: 0.5465 - val_accuracy: 0.7286 - val_loss: 0.5529
Epoch 5/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7332 - loss: 0.5432 - val_accuracy: 0.7310 - val_loss: 0.5499
Epoch 6/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7295 - loss: 0.5450 - val_accuracy: 0.7273 - val_loss: 0.5537
Epoch 7/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7326 - loss: 0.5438 - val_accuracy: 0.7311 - val_loss: 0.5499
Epoch 8/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7353 - loss: 0.5416 - val_accuracy: 0.